## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
from pathlib import Path
import json
import pandas as pd
import requests
import gmaps
from config import g_key

# Configure gmaps
# Load key

key_path = Path("resources") / "keys.json"
if key_path.is_file():
    with open(key_path, "r") as keys:
        g_key = json.load(keys)[0]["google"]

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
0,10,Tuktoyaktuk,CA,53.60,broken clouds,69.4541,-133.0374,Smitty's Bed and Breakfast
1,13,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel
2,14,Sitka,US,67.89,overcast clouds,57.0531,-135.3300,Sitka Hotel
3,17,Nikolskoye,RU,64.29,overcast clouds,59.7035,30.7861,Tourist House - Sablino
4,26,Busselton,AU,53.82,clear sky,-33.6500,115.3333,Esplanade Hotel


In [4]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}, {MaxTemp} °F</dd>
</dl>
"""


# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lon"]]

In [5]:
# 4a. Add a marker layer for each city to the map.
max_temp = vacation_df["MaxTemp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Santo Tirso"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Santo Tirso"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Villaviciosa"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Muros"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Carballo"] 

In [17]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start.to_numpy()[0][5:7]
end = vacation_end.to_numpy()[0][5:7]
stop1 = vacation_stop1.to_numpy()[0][5:7]
stop2 = vacation_stop2.to_numpy()[0][5:7]
stop3 = vacation_stop3.to_numpy()[0][5:7]
start, end, stop1, stop2, stop3

(array([41.3426, -8.4775], dtype=object),
 array([41.3426, -8.4775], dtype=object),
 array([43.4813, -5.4357], dtype=object),
 array([42.7762, -9.0603], dtype=object),
 array([43.213, -8.691], dtype=object))

In [19]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
# Latitude-longitude pairs
santo_tirso = start
villaviciosa = stop1
muros = stop2
carballo = stop3

fig = gmaps.figure()
trip = gmaps.directions_layer(
    santo_tirso, 
    santo_tirso, 
    waypoints=[villaviciosa, muros, carballo],
    travel_mode="DRIVING")
fig.add_layer(trip)
fig


Figure(layout=FigureLayout(height='420px'))

In [21]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start, 
        vacation_stop1, 
        vacation_stop2, 
        vacation_stop3, 
        vacation_end
    ],
        ignore_index=True
)
itinerary_df

,City_ID,City,Country,MaxTemp,Current Description,Lat,Lon,Hotel Name
0,52,Santo Tirso,PT,67.46,clear sky,41.3426,-8.4775,Cidnay Santo Tirso - Charming Hotel & Executiv...
1,1652,Villaviciosa,ES,69.40,few clouds,43.4813,-5.4357,Hotel Arcea Villaviciosa
2,854,Muros,ES,63.09,clear sky,42.7762,-9.0603,A Casa da Fonte
3,331,Carballo,ES,67.42,broken clouds,43.2130,-8.6910,Hotel Mar De Queo
4,52,Santo Tirso,PT,67.46,clear sky,41.3426,-8.4775,Cidnay Santo Tirso - Charming Hotel & Executiv...


In [22]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}, {MaxTemp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lon"]]

In [25]:
# 11a. Add a marker layer for each city to the map.
max_temp = itinerary_df["MaxTemp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(
    locations, weights=max_temp,dissipating=False,
    max_intensity=300, point_radius=4
)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))